In [46]:
import pandas as pd
import dask.dataframe as dd
from dask_ml import preprocessing
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [47]:
target = ["TARGET", "Target", "target", "CHURN", "Churn", "churn", "RESULT", "Result", "result"]

In [48]:
# Obtiene una lista con las columnas categoricas a partir de una diferencia de conjuntos
def getCategoricalColumns(df):
    cols = set(df.columns)
    numColumns = set(df._get_numeric_data().columns)
    
    return list(cols - numColumns)

# Convierte las variables categoricas en numericas en base a label encoding
def getNumericDataset(df):
    categoricalColumns = getCategoricalColumns(df)
    numDataset = df.drop(columns= categoricalColumns)
    categoricalDataset = df[categoricalColumns]

    le = preprocessing.LabelEncoder()
    for i in categoricalColumns:
        categoricalDataset[i] = le.fit_transform(categoricalDataset[i])
    
    for i in categoricalColumns:
        numDataset[i] = categoricalDataset[i]
    
    return numDataset

In [49]:
def scaler(df):
    scaler = StandardScaler()
    return scaler.fit_transform(df)

In [105]:
def reduce_csv(csv_file):
    df = dd.read_csv(csv_file)
    result = df[df.columns.intersection(target)]
    df = df.drop(columns= target + ["Unnamed: 0"], errors='ignore')
    
    x = getNumericDataset(df)
    columns = list(x.columns)
    x = scaler(x)
    
    pca = PCA(n_components=.8, svd_solver='full')
    pca.fit(x)

    reduced = []
    for i in range(0, len(pca.explained_variance_ratio_)):
        print(i)
        reduced.append(columns[i])   

    result[reduced] = df[reduced]

    result.to_csv(csv_file, index=False, single_file=True)


In [106]:
df, n, b =reduce_csv("/home/alt9193/Documents/IA/DeepLearningBackend/examples/telecom_churn_me.csv")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [107]:
b

['YEAR_JOINED',
 'CURRENT_YEAR',
 'BILL_AMOUNT',
 'PAID_AMOUNT',
 'PAYMENT_TRANSACTIONS',
 'PARTY_REV',
 'PREPAID_LINES',
 'POSTPAID_LINES',
 'OTHER_LINES',
 'LINE_REV',
 'MOUS_TO_LOCAL_MOBILES',
 'MOUS_FROM_LOCAL_MOBILES',
 'MOUS_TO_LOCAL_LANDLINES',
 'MOUS_FROM_LOCAL_LANDLINES',
 'MOUS_TO_INT_NUMBER']

In [108]:
df

,TARGET,YEAR_JOINED,CURRENT_YEAR,BILL_AMOUNT,PAID_AMOUNT,PAYMENT_TRANSACTIONS,PARTY_REV,PREPAID_LINES,POSTPAID_LINES,OTHER_LINES,LINE_REV,MOUS_TO_LOCAL_MOBILES,MOUS_FROM_LOCAL_MOBILES,MOUS_TO_LOCAL_LANDLINES,MOUS_FROM_LOCAL_LANDLINES,MOUS_TO_INT_NUMBER
npartitions=3,,,,,,,,,,,,,,,,
,int64,int64,int64,float64,float64,int64,float64,int64,int64,int64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [109]:
n

15

In [110]:
len(df.columns)

16